In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm
df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']

# Precision 점수를 저장할 리스트 초기화
precision_scores = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Evaluating Random States'):
    X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=state)
    
    lr_model = LogisticRegression(max_iter=10000, random_state=state)
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    precision_scores.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_scores_df = pd.DataFrame(precision_scores)

# DataFrame 출력 설정 변경
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 결과 출력
print(precision_scores_df)

Evaluating Random States:   0%|          | 0/100 [00:00<?, ?it/s]

    Random_State  Precision
0              1   0.563189
1              2   0.501371
2              3   0.562540
3              4   0.572585
4              5   0.497962
5              6   0.553766
6              7   0.541905
7              8   0.558761
8              9   0.586021
9             10   0.552355
10            11   0.524856
11            12   0.503993
12            13   0.545259
13            14   0.547570
14            15   0.524244
15            16   0.550343
16            17   0.579120
17            18   0.609423
18            19   0.546932
19            20   0.569444
20            21   0.527351
21            22   0.536693
22            23   0.532077
23            24   0.532478
24            25   0.536968
25            26   0.583044
26            27   0.492832
27            28   0.591320
28            29   0.596758
29            30   0.555579
30            31   0.582267
31            32   0.582942
32            33   0.568606
33            34   0.551031
34            35   0

In [4]:
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)

In [ ]:
precision_scores_df_sorted

In [7]:
precision_scores_df_sorted = precision_scores_df.sort_values(by='Precision', ascending=False)

In [8]:
precision_scores_df_sorted 

,Random_State,Precision
65,66,0.617576
17,18,0.609423
69,70,0.602770
83,84,0.601946
28,29,0.596758
27,28,0.591320
49,50,0.590561
55,56,0.589582
60,61,0.586111
8,9,0.586021
